# Exploring Open Web RX waterfall data
An exercise in data aquisition and signals processing.

## Metadata collection

First off we will select a site and get some metadata to assess if we can use it.

In [ ]:
# Lets explore some open web rx data.
# we could visit and scrape https://www.receiverbook.de/?type=openwebrx for links... 
# but lets leave that for later and go straight to a single source. 

import json
import httpx

# "randomly" chosen openwebrx site
site = "https://2.websdr.jestok.com"

# Let's fetch the features of the site first
result = httpx.get(site + "/api/features")
result.raise_for_status()
features = result.json()

In [ ]:
# Now lets get some metrics from the openwebrx device to see how its doing.
result = httpx.get(site + "/metrics.json")
result.raise_for_status()
metrics = result.json()
# Note that if the user count is high, we might want to consider a different site.

In [ ]:
# Lets get some metadata to see where the receiver is located. (asl = above sea level)
result = httpx.get(site + "/status.json")
result.raise_for_status()
status = result.json()
status["receiver"]

Okay, the status json should look something line this:
```
{'name': 'SR4DON OLSZTYN/POLAND 17m-160m',
 'admin': 'sdr@jestok.com',
 'gps': {'lat': 53.7832485938901, 'lon': 20.4547299031047},
 'asl': 134,
 'location': 'Olsztyn, Poland'}
```
So it looks like this particular receiver is located in Poland so lets move on with that.

## Waterfall data

Lets have a look at what the web viewer is getting when opening up the site. 
When you visit an openwebrx page you are greeted by this wonderfull (near) real-time waterfall.
So lets have a look at that datastream, shall we 

In [ ]:
# Get 20 items from the websocket stream
from websockets.asyncio.client import connect

messages = []
async with connect("wss://2.websdr.jestok.com/ws/") as ws:
    await ws.send("SERVER DE CLIENT client=lib/FrequencyDisplay.js type=receiver")
    for _ in range(20):
        msg = await ws.recv()
        messages.append(msg)

print(len(messages))

In [ ]:
messages

Great scott! its really quite fast... we get a bunch more metadata like the the position of the receiver, some dial frequencies, bands etc.. Then we get a bunch of binary data! cool... but what is it? 
```
b'\x01\xff\xff\xff\xff\x0e\x08\x08\x08\x08\x88\x00\x08\x08...
```

well, we did get a hint in the config dictionary that came before the data: 
```
"waterfall_scheme": "TeejeezWaterfall", "tuning_precision": 2, "fft_size": 16384, "audio_compression": "adpcm", "waterfall_auto_levels": {"min": 3.0, "max": 10.0}
```

In [ ]:
# Select a binary string to proceed with
message = messages[18]
message

In [ ]:
# Load the config and get the start frequency
config = json.loads(messages[6])
center_freq = config["value"]["center_freq"]
start_freq = config["value"]["start_freq"]
bandwidth = 2*(center_freq - start_freq)
stop_freq = start_freq + bandwidth
print(f"{start_freq} - {bandwidth} - {stop_freq}")

In [ ]:
# We want to get some data from this 
info = json.loads(messages[2])
fft_compression = info["value"]["fft_compression"]
fft_size = info["value"]["fft_size"]
position = info["value"]["receiver_gps"]
info_profile = json.loads(messages[6])
profile = info_profile["value"]
profile

In [ ]:
# select one of the data rows for processing and load it into an array
import numpy as np
data_array = np.frombuffer(message[1:], dtype=np.uint8)
data_array[25:50]
# (Psst: the first byte is the type of message, the second byte is the length of the data, and the rest is the actual data)

What the... what on earth is this? 
8, 129, 136,  13, 116, 233,   1,  24, 128,  26,  17, 169,   3,   122,  48, 159,   0, 136, 128,   8, 128,   2, 136, 184,  44

Exciting! But the hint came in the metadata before: "audio_compression": "adpcm"
This is an adaptive differential pulse-code modulation. A very effivient supposedly lossless compression for a digital signal.
Well thats cool! Right?!... just me? okaaay... whatever, you're the one reading this. lets parse it.  

In [ ]:
# right about now is a goot time to start timing how long it takes to process the data.
from time import perf_counter_ns

In [ ]:
# The bitstream is a split-band ADPCM stream. So we have to split the bytes into 4 bit nibbles. 
def split_nibbles(byte_array):
    high_nibbles = np.bitwise_and(byte_array, 0xF0) >> 4
    low_nibbles = np.bitwise_and(byte_array, 0x0F)
    return high_nibbles, low_nibbles

tic = perf_counter_ns()
higher, lower = split_nibbles(data_array)
toc = perf_counter_ns()
print(f"Split nibbles took {(toc - tic)/1000} microseconds")

In [ ]:
# So now that the bytes have been separated into high and low nibbles, we can interleave them.
tic = perf_counter_ns()
interleaved = np.array([val for pair in zip(lower, higher) for val in pair], dtype=np.uint8)
toc = perf_counter_ns()
print(f"interleaving took {(toc - tic)/1000} microseconds")
interleaved

In [ ]:
# So thats the raw ADPCM data in the array, its a bunch of integer numbers ofcourse but lets plot a histogram to get a sense of what kind of numbers there are in there.
import matplotlib.pyplot as plt 
plt.figure()
plt.hist(interleaved, bins=255)
plt.show()

So this is neat, we can see we have an interesting distriubtion of values no higher than 15 which makes sense for a 4 bit byte. 
PCM, LPCM and ADPCM is commonly used in audio encoding because it is space efficient and very computationally fast to decode.
In short, ADPCM uses two internal lookup tables to get a step size, and then applies the difference between the step size and the previously decoded value to obtain a new value. 

In [ ]:
# I found a python library that can decode ADPCM data, but its not packaged.
# I've downloaded the adpcm.py file and added it to the root of this project, which is what i'll be using here now.
# The adpcm library is from https://github.com/karnwatcharasupat/pyADPCM
from adpcm import ADPCM

# perform an adpmc decode operation on the interleaved data...add()
# Remove the first 10 numbers from the list and devide every number by 100

adpcm = ADPCM()

fft = np.array(adpcm.decode(interleaved), dtype=np.float32)[10:]/100
print(len(fft), fft)

# If you reference the above metadata from the websocket stream you can see that the fft size is supposed to be 16384, which is why we remove the first 10 numbers as they are essentially an encoding artifact.
# The value is 

In [ ]:
frequency_range = np.linspace(start_freq, stop_freq, fft_size)/1000000

In [ ]:
# Now plot the completed data array and you should see a spectral plot
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline

fig, ax = plt.subplots(figsize=(16,4))
plt.ylabel("dBm")
line1, =ax.plot(frequency_range, fft)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter("%f MHz"))
plt.show()  

In [ ]:
# All togheter now...add()
import time

tic = time.perf_counter()
data_array = np.frombuffer(message[1:], dtype=np.uint8)
higher, lower = split_nibbles(data_array)
interleaved = np.array([val for pair in zip(lower, higher) for val in pair], dtype=np.uint8)
fft = np.array(adpcm.decode(interleaved), dtype=np.float32)[10:]/100
toc = time.perf_counter()
print(toc - tic)

So thats... fairly fast now that i started using uv. (before it was like 0.12 ish seconds)
But not fast enough to graph in real time with python so i decided to write the modules for nibbling and adpcm in rust to speed things up a bit. 

(you might have to add the extra modules)

```
uv pip install -e nibbler
uv pip install -e radpcm
```

In [ ]:
import numpy as np
import time
import nibbler
import radpcm
def decode_fft_bitstream(message: bytes) -> np.ndarray[np.int16]: 
    # strip the first character and load into array
    interleaved = nibbler.split_and_interleave_nibbles(message)
    fft = radpcm.decode(interleaved)[10:]/100
    return fft
tic = time.perf_counter()
rfft = decode_fft_bitstream(message[1:])
toc = time.perf_counter()
print(toc - tic)

So that is a performance boost of two orders of magninutde. which i believe would suffice for most use cases i can come up with right now. 


# Realtime processing
Okay that was all mostly theoretical stuff, lets look at some graphs.

In [ ]:
# Re-import just so we dont have to run the whole script again
import numpy as np
import nibbler
import radpcm

import matplotlib.pyplot as plt
import httpx
import websockets

In [ ]:
import json
def handle_info(message: str, receiver: dict) -> dict:
    info = json.loads(message)
    info_type = info["type"]
    match info_type:
        case "config" | "receiver_details":
            for key, value in info["value"]: 
                receiver[key] = value
            return receiver
        case "cpuusage" | "temp":
            receiver[info_type] = info["value"]
            return receiver
        case _:
            return receiver

In [ ]:
# Configure site url from dashboard. 
# Get receiver info on demand. 

site = "2.websdr.jestok.com"
result = httpx.get("https://" + site + "/status.json")
result.raise_for_status()
status = result.json()
receiver = status["receiver"]
receiver

In [ ]:
# Connect to the websocket stream and listen for messages.
from websockets.asyncio.client import connect
n = 1000  # Number of messages to receive
fft_data = []

async with connect("wss://" + site + "/ws/", open_timeout=None, ping_timeout=None) as ws:
    await ws.send("SERVER DE CLIENT client=lib/FrequencyDisplay.js type=receiver")
    await ws.send('{"type":"connectionproperties","params":{"nr_enabled":false,"nr_threshold":0}}')
    # await ws.send('{"type":"dspcontrol","params":{"low_cut":150,"high_cut":2750,"offset_freq":1859450,"mod":"usb","dmr_filter":3,"audio_service_id":0,"squelch_level":-150,"secondary_mod":false}}')
    # await ws.send('{"type":"dspcontrol","action":"start"}')
    for _ in range(n):
        message = await ws.recv()
        match message[0]:
            case 1:
                fft = decode_fft_bitstream(message[1:])
                fft_data.append(fft)
                # What to do with the decoded fft...
                # Send it to a function that updates the plot in a dashboard. 
                # Plotly seems like a good option.
            # case "{":
            #     # Update receiver information.
            #     receiver = handle_info(message, receiver)
            # case "C":
            #     print(message)
            case _:
                # print("Message not handled: ", message)
                pass

    
print(f"Received {len(fft_data)} messages with FFT data.")

# Fair warning: With 2000 samples its gonna take 4 minutes to collect.

In [ ]:
np.array(fft_data)

In [ ]:
# So now lets finally have a look at the waterfall plot of the data we collected.

import plotly.express as px
fig = px.imshow(
    np.array(fft_data), 
    aspect="auto", 
    origin="lower", 
    labels={
        "x": "Frequency (MHz)",
        "y": "Sample",
        "color": "dBm",
    }, 
    title=f"Waterfall plot of {site} FFT data"
)
x_tics = np.arange(np.array(fft_data).shape[1]) + start_freq/1000000  # Convert to MHz
fig.update_xaxes(tickvals=x_tics, ticktext=[f"{x:.2f} MHz" for x in x_tics])

fig.show()

## TODO

- Find a way to create a dashboard
- make the dashboard interactive to enter, start and stop a websocket connection.
- make a realtime spectrogram
- create a self updating waterfall plot (fixed window)

and once thats done. 
- Create a database to keep the data in.
- Create an ingest tool that loads into the database
- Create a way for the dashboard to query the waterfall database and produce an image.
- Join that new waterfall viz with the updating one using a subscription to the waterfall stream. 

In [ ]:
# plotly
import plotly.express as px
# Resample size (e.g., take max of every 100 points)
chunk_size = 40
data = fft
# Reshape and take the max
reduced_data_max = data[:len(data) // chunk_size * chunk_size].reshape(-1, chunk_size).max(axis=1)
px.line(y=reduced_data_max)
